In [1]:
# Example 5: RAG - Retrieval-Augmented Generation
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import os

# Initialize components
llm = ChatOpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    model="gpt-3.5-turbo",
    temperature=0.1  # Lower temperature for more factual responses
)

embeddings = OpenAIEmbeddings(api_key=os.getenv("OPENAI_API_KEY"))

print("📚 RAG System: Building AI That Knows Your Documents")
print("="*60)

# Step 1: Create sample documents (in a real scenario, you'd load from files)
sample_documents = [
    """
    Company Policy: Remote Work Guidelines
    
    Effective Date: January 1, 2024
    
    Our company supports flexible work arrangements to promote work-life balance and productivity.
    
    Remote Work Eligibility:
    - Employees must have been with the company for at least 6 months
    - Role must be suitable for remote work (approved by department head)
    - Employee must have a dedicated workspace and reliable internet
    
    Remote Work Schedule:
    - Full-time remote: Up to 5 days per week from home
    - Hybrid: 2-3 days in office, remainder remote
    - Flexible hours between 7 AM - 6 PM local time
    
    Equipment and Technology:
    - Company provides laptop and necessary software licenses
    - Employees responsible for home internet and workspace setup
    - IT support available for company equipment
    
    Performance and Communication:
    - Weekly one-on-one meetings with direct supervisor
    - Participate in all team meetings via video conference
    - Maintain regular communication through Slack during business hours
    - Quarterly performance reviews remain unchanged
    
    Security Requirements:
    - Use VPN for all company network access
    - Enable two-factor authentication on all accounts
    - Do not work from public spaces when handling sensitive information
    """,
    
    """
    Employee Benefits Guide
    
    Health Insurance:
    - PPO and HMO options available
    - Company pays 80% of premiums for employee
    - Family coverage available (employee pays 40% of additional cost)
    - $500 annual health and wellness reimbursement
    
    Retirement Plan:
    - 401(k) with 4% company matching (100% vested after 2 years)
    - Financial planning consultations available quarterly
    - Automatic enrollment at 3% contribution rate
    
    Paid Time Off:
    - 15 days PTO for years 1-3
    - 20 days PTO for years 4-7  
    - 25 days PTO for years 8+
    - 10 paid holidays annually
    - Unlimited sick leave (manager approval for 5+ consecutive days)
    
    Professional Development:
    - $2,000 annual learning budget per employee
    - Conference attendance supported (pre-approval required)
    - LinkedIn Learning and Coursera subscriptions provided
    - Mentorship program available
    
    Other Benefits:
    - Flexible spending accounts (FSA) for healthcare and dependent care
    - Employee assistance program (EAP) for counseling and support
    - Company-sponsored team events and happy hours
    - Discounted gym memberships at partner locations
    """,
    
    """
    IT Security Policy
    
    Password Requirements:
    - Minimum 12 characters with mix of letters, numbers, and symbols
    - Changed every 90 days
    - Cannot reuse last 12 passwords
    - Use company-approved password manager
    
    Data Classification:
    - Public: Can be shared freely (marketing materials, job postings)
    - Internal: Company information not meant for external sharing
    - Confidential: Customer data, financial information, strategic plans
    - Restricted: Legal documents, HR records, security credentials
    
    Email Security:
    - Be suspicious of unexpected attachments or links
    - Verify sender identity for sensitive requests
    - Use encryption for confidential information
    - Report phishing attempts to IT immediately
    
    Device Management:
    - All company devices must be encrypted
    - Install security updates within 48 hours of release
    - Report lost or stolen devices immediately
    - Personal use of company devices is acceptable for reasonable personal activities
    
    Incident Response:
    - Report security incidents to IT within 1 hour of discovery
    - Do not attempt to fix security issues independently
    - Preserve evidence by avoiding further system use
    - Cooperate fully with security investigations
    """
]

# Step 2: Process documents into chunks
print("🔄 Step 1: Processing documents...")

# Create text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,    # Size of each chunk
    chunk_overlap=200,  # Overlap between chunks to maintain context
    length_function=len
)

# Split documents into chunks
all_chunks = []
for i, doc_text in enumerate(sample_documents):
    chunks = text_splitter.split_text(doc_text)
    for chunk in chunks:
        all_chunks.append(chunk)

print(f"✅ Created {len(all_chunks)} document chunks")

# Step 3: Create embeddings and vector store
print("🔄 Step 2: Creating vector embeddings...")

try:
    # Create vector store
    vectorstore = Chroma.from_texts(
        texts=all_chunks,
        embedding=embeddings,
        persist_directory="./chroma_db"  # Saves the database locally
    )
    print("✅ Vector store created successfully")
except Exception as e:
    print(f"❌ Error creating vector store: {e}")
    print("💡 Make sure you have chromadb installed: !pip install chromadb")

# Step 4: Create retrieval system
print("🔄 Step 3: Setting up retrieval system...")

# Create a retriever from the vector store
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}  # Return top 3 most similar chunks
)

# Create a custom prompt for the QA system
qa_prompt_template = """
You are a helpful assistant answering questions about company policies and procedures.

Use the following pieces of context to answer the question. If you don't know the answer based on the context provided, say that you don't have that information in the available documents.

Context:
{context}

Question: {question}

Please provide a clear, accurate answer based on the context above. If relevant, mention the specific policy or section you're referencing.

Answer:
"""

qa_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=qa_prompt_template
)

# Create the QA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff", #just concatenates top-k (here taken 3) chuncks. alternatives include "map_reduce", "refine".
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": qa_prompt}
)

print("✅ RAG system ready!")

# Step 5: Test the system with various questions
test_questions = [
    "What are the requirements for remote work eligibility?",
    "How much PTO do I get after working here for 5 years?",
    "What should I do if I receive a suspicious email?",
    "Does the company provide any learning budget?",
    "What are the password requirements?",
    "Can I work from a coffee shop?",
    "What happens if I lose my company laptop?",
    "If I have joined 11 months back, how much PTO do I have?",
    "How much does the company contribute to health insurance?"
]

print("\n🎯 Testing RAG System with Sample Questions:")
print("="*60)

for i, question in enumerate(test_questions, 1):
    print(f"\n❓ Question {i}: {question}")
    
    try:
        # Get answer from RAG system
        result = qa_chain({"query": question})
        
        print(f"🤖 Answer: {result['result']}")
        
        # Show which documents were used (for transparency)
        print(f"\n📋 Sources used:")
        for j, doc in enumerate(result['source_documents']):
            preview = doc.page_content[:100].replace('\n', ' ')
            print(f"   Source {j+1}: {preview}...")
        
        print("-" * 50)
        
    except Exception as e:
        print(f"❌ Error: {e}")



📚 RAG System: Building AI That Knows Your Documents
🔄 Step 1: Processing documents...
✅ Created 6 document chunks
🔄 Step 2: Creating vector embeddings...
✅ Vector store created successfully
🔄 Step 3: Setting up retrieval system...
✅ RAG system ready!

🎯 Testing RAG System with Sample Questions:

❓ Question 1: What are the requirements for remote work eligibility?


/var/folders/55/svxq6kt93z3147gppt0gqj580000gn/T/ipykernel_44534/1657854010.py:223: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": question})


🤖 Answer: To be eligible for remote work, employees must have been with the company for at least 6 months, have a role suitable for remote work approved by their department head, and have a dedicated workspace with reliable internet. These requirements are outlined in the "Remote Work Eligibility" section of the Company Policy: Remote Work Guidelines.

📋 Sources used:
   Source 1: Company Policy: Remote Work Guidelines      Effective Date: January 1, 2024      Our company support...
   Source 2: Company Policy: Remote Work Guidelines      Effective Date: January 1, 2024      Our company support...
   Source 3: Company Policy: Remote Work Guidelines      Effective Date: January 1, 2024      Our company support...
--------------------------------------------------

❓ Question 2: How much PTO do I get after working here for 5 years?
🤖 Answer: After working at the company for 5 years, you would be entitled to 20 days of Paid Time Off (PTO) per year. This information can be found in the Pai

In [2]:
# Step 6: Advanced RAG - Custom scoring and filtering
print("\n\n🔬 Advanced RAG: Custom Retrieval with Scoring")
print("="*50)

def advanced_rag_query(question, similarity_threshold=0.7):
    """
    Advanced RAG function that shows similarity scores and filters results
    """
    try:
        # Get documents with scores
        docs_and_scores = vectorstore.similarity_search_with_score(question, k=5)
        
        print(f"🔍 Searching for: '{question}'")
        print(f"📊 Similarity Results:")
        
        relevant_docs = []
        for i, (doc, score) in enumerate(docs_and_scores):
            print(f"   Document {i+1}: Score {score:.3f}")
            if score < similarity_threshold:  # Lower score = more similar
                relevant_docs.append(doc)
                preview = doc.page_content[:80].replace('\n', ' ')
                print(f"   ✅ RELEVANT: {preview}...")
            else:
                print(f"   ⚪ Below threshold")
        
        if relevant_docs:
            # Create context from relevant documents
            context = "\n\n".join([doc.page_content for doc in relevant_docs])
            
            # Generate answer using the LLM
            response = llm.invoke(f"""
            Based on this context:
            {context}
            
            Question: {question}
            
            Provide a clear, accurate answer. If the context doesn't contain enough information, say so.
            """)
            
            print(f"\n🤖 Generated Answer:")
            print(response.content)
        else:
            print("❌ No sufficiently relevant documents found")
            
    except Exception as e:
        print(f"❌ Error in advanced RAG: {e}")

# Test advanced RAG
advanced_questions = [
    "What's the company's policy on working from public places?",
    "How often do I need to change my password?"
]

for question in advanced_questions:
    print("\n" + "="*60)
    advanced_rag_query(question)

print("\n✅ RAG (Retrieval-Augmented Generation) demonstration complete!")



🔬 Advanced RAG: Custom Retrieval with Scoring

🔍 Searching for: 'What's the company's policy on working from public places?'
📊 Similarity Results:
   Document 1: Score 0.382
   ✅ RELEVANT: Company Policy: Remote Work Guidelines      Effective Date: January 1, 2024     ...
   Document 2: Score 0.382
   ✅ RELEVANT: Company Policy: Remote Work Guidelines      Effective Date: January 1, 2024     ...
   Document 3: Score 0.382
   ✅ RELEVANT: Company Policy: Remote Work Guidelines      Effective Date: January 1, 2024     ...
   Document 4: Score 0.447
   ✅ RELEVANT: Performance and Communication:     - Weekly one-on-one meetings with direct supe...
   Document 5: Score 0.447
   ✅ RELEVANT: Performance and Communication:     - Weekly one-on-one meetings with direct supe...

🤖 Generated Answer:
The company's policy on working from public places is that employees should not work from public spaces when handling sensitive information.

🔍 Searching for: 'How often do I need to change my passwor